
<BR>
<CENTER>
<FONT SIZE = 4, COLOR="#2C32C3">
<BR><B>Northeastern University</B>
<BR><B><font size=4, color=black>Husky-3,</BR>
ALY6010 Prob Theory and Intro Stats</B>
 <BR> Motor Vehicle Collision Analysis in New York(2017-2023)
<BR> Final Project
<BR> Rohit Kosamkar, Sapna Chavan, Leena Doultani, Soumya Guliyan, Aliha Ansari, Baudhik Kevadiya
<BR><font size=4, color=dark red> Date: 03.24.2024
</FONT>
</CENTER>

In [2]:
library(dplyr)
library(arrow)
library(ggplot2)
library(corrplot)
library(caret)
library(car)
library(fastDummies)
set.seed(123)

In [3]:
# Reading Dataset 
crash_data <- read_parquet("crash_data.parquet")
dim(crash_data)

[1] 1112224      27

In [4]:
# let's have a view of data
head(crash_data[,10:15])

,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed
,<int>,<int>,<int>,<int>,<int>,<int>
1,0,0,0,0,2,0
2,0,0,0,0,1,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0


In [5]:
str(crash_data)

'data.frame':	1112224 obs. of  27 variables:
 $ X                            : int  1 2 3 4 5 6 7 8 9 10 ...
 $ crash_date                   : chr  "2021-09-11" "2022-03-26" "2022-06-29" "2021-09-11" ...
 $ crash_time                   : chr  "2:39" "11:45" "6:55" "9:35" ...
 $ borough                      : chr  "" "" "" "BROOKLYN" ...
 $ zip_code                     : int  NA NA NA 11208 11233 NA NA 10475 11207 10017 ...
 $ latitude                     : num  NA NA NA 40.7 40.7 ...
 $ longitude                    : num  NA NA NA -73.9 -73.9 ...
 $ number_of_persons_injured    : int  2 1 0 0 0 0 0 2 0 0 ...
 $ number_of_persons_killed     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ number_of_pedestrians_injured: int  0 0 0 0 0 0 0 0 0 0 ...
 $ number_of_pedestrians_killed : int  0 0 0 0 0 0 0 0 0 0 ...
 $ number_of_cyclist_injured    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ number_of_cyclist_killed     : int  0 0 0 0 0 0 0 0 0 0 ...
 $ number_of_motorist_injured   : int  2 1 0 0 0 0 0 2 0 0 ...
 $ num

In [6]:
colSums(is.na(crash_data))

X                    crash_date 
                            0                             0 
                   crash_time                       borough 
                            0                             0 
                     zip_code                      latitude 
                       393070                         81212 
                    longitude     number_of_persons_injured 
                        81212                             0 
     number_of_persons_killed number_of_pedestrians_injured 
                            0                             0 
 number_of_pedestrians_killed     number_of_cyclist_injured 
                            0                             0 
     number_of_cyclist_killed    number_of_motorist_injured 
                            0                             0 
    number_of_motorist_killed contributing_factor_vehicle_1 
                            0                             0 
contributing_factor_vehicle_2 contributing_factor_vehicle_3 
                            0                             0 
                 collision_id           vehicle_type_code_1 
                            0                             0 
          vehicle_type_code_2           vehicle_type_code_3 
                            0                             0 
                        month                          year 
                            0                             0 
                 total_injury                total_casualty 
                            0                             0 
        total_injuries_deaths 
                            0

In [7]:
names(crash_data)

[1] "X"                             "crash_date"                   
 [3] "crash_time"                    "borough"                      
 [5] "zip_code"                      "latitude"                     
 [7] "longitude"                     "number_of_persons_injured"    
 [9] "number_of_persons_killed"      "number_of_pedestrians_injured"
[11] "number_of_pedestrians_killed"  "number_of_cyclist_injured"    
[13] "number_of_cyclist_killed"      "number_of_motorist_injured"   
[15] "number_of_motorist_killed"     "contributing_factor_vehicle_1"
[17] "contributing_factor_vehicle_2" "contributing_factor_vehicle_3"
[19] "collision_id"                  "vehicle_type_code_1"          
[21] "vehicle_type_code_2"           "vehicle_type_code_3"          
[23] "month"                         "year"                         
[25] "total_injury"                  "total_casualty"               
[27] "total_injuries_deaths"

### Feature Engineering

In [8]:
# cleaning crash time
crash_data$crash_time <- gsub("^([0-9]{1,2}):([0-9]{2}).*$", "\\1:\\2:00", crash_data$crash_time)
# Standardizing the format of crash_time (assuming it's in HH:MM format)
crash_data$crash_datetime <- as.POSIXlt(paste(crash_data$crash_date, crash_data$crash_time), format = "%Y-%m-%d %H:%M:%S")

# creating new column Time period based on rush hour at morning & evening
crash_data$time_period <- factor(ifelse(format(crash_data$crash_datetime, "%H:%M") >= "08:00" & 
                                          format(crash_data$crash_datetime, "%H:%M") <= "10:00", 
                                        "Morning Rush Hour",
                                        ifelse(format(crash_data$crash_datetime, "%H:%M") >= "17:00" &
                                                 format(crash_data$crash_datetime, "%H:%M") <= "19:00", 
                                               "Evening Rush Hour", 
                                               "Non Rush Hour")), 
                                 levels = c("Morning Rush Hour", "Evening Rush Hour", "Non Rush Hour"))


In [9]:
# Adding season across new york 

# Creating a vector with month-wise season mapping
season_mapping <- c(Spring = c(3, 4, 5),
                    Summer = c(6, 7, 8),
                    Fall = c(9, 10, 11),
                    Winter = c(12, 1, 2))

# Function to map month to season
get_season <- function(month) {
  for (season in names(season_mapping)) {
    if (month %in% season_mapping[[season]]) {
      return(season)
    }
  }
}

crash_data$Season <- sapply(crash_data$month, get_season)
crash_data$Season <- gsub("[0-9]", "", crash_data$Season)


In [10]:
unique(crash_data$Season)

[1] "Fall"   "Spring" "Summer" "Winter"

In [11]:
#  grouping contributing factors  

grouped_factors <- case_when(
  grepl("Aggressive Driving|Rage|Following Too Closely|Inexperience|Passing|Turning|Unsafe Lane|Reaction|Inattention|Pedestrian|Failure to Yield|Illness|Lost Consciousness|Fell Asleep|Animals|Fatigued|Physical Disability|Eating or Drinking|Texting|Listening", crash_data$contributing_factor_vehicle_1) ~ "Human Error",
  grepl("Brakes|Steering|Accelerator|Tire|Headlights|Windshield|Shoulders|Tow Hitch", crash_data$contributing_factor_vehicle_1) ~ "Vehicle-related Issues",
  grepl("Pavement|Glare|Obstruction|Lighting", crash_data$contributing_factor_vehicle_1) ~ "Environmental Factors",
  grepl("Alcohol|Drugs|Prescription Medication", crash_data$contributing_factor_vehicle_1) ~ "Substance Use",
  grepl("Cell Phone|Navigation|Electronic Device", crash_data$contributing_factor_vehicle_1) ~ "Technology-related",
  TRUE ~ "Miscellaneous"
)

crash_data$grouped_factor_vehicle_contribution <- grouped_factors

In [12]:
# Grouping vehicle type
crash_data$vehicle_type_code_1 <- tolower(crash_data$vehicle_type_code_1)


category_mapping <- list(
  "Passenger Vehicles" = c(
    "sedan", "sport utility vehicle", "convertible", "van", "pick-up truck", "motorcycle", 
    "bike", "e-bike", "motorscooter", "e-scooter", "moped", "scooter", "taxi","motorbike"
  ),
  "Commercial Vehicles" = c(
    "dump", "tanker", "bus", "box truck", "garbage or refuse", "tractor truck diesel","beverage truck",
    "flat bed", "tow truck / wrecker", "ambulance", "trailer", "school bus", "fire truck"
  ),
  "Other Vehicles" = c(
     "utility", "ï¿½mbu", "pk", "carry all", "fdny ambul", "3-door", 
    "semi trail", "tow truck", "unknown", "trailer", "lunch wagon", "flat rack", 
    "unk", "uspcs", "tractor truck gasoline", "garbage tr", "minicycle",  
    "lift boom", "cater", "pkup"
  )
)

# Function to categorize vehicle types
categorize_vehicle <- function(vehicle_type) {
  vehicle_type <- tolower(vehicle_type)  # Convert to lowercase
  for (cat in names(category_mapping)) {
    if (vehicle_type %in% category_mapping[[cat]]) {
      return(cat)
    }
  }
  return("Other")
}

crash_data <- crash_data %>% 
  mutate(vehicle_category = sapply(vehicle_type_code_1, categorize_vehicle))


In [13]:
# creating feature pedestrian involved
crash_data <- crash_data %>% 
  mutate(pedestrian_involved = ifelse(number_of_pedestrians_injured > 0 | number_of_pedestrians_killed > 0, 1, 0))


In [14]:
# creating feature cyclist involved
crash_data <- crash_data %>% 
  mutate(cyclist_involved = ifelse(number_of_cyclist_injured > 0 | number_of_cyclist_killed > 0, 1, 0))


In [15]:
# creating feature motorist involved
crash_data <- crash_data %>% 
  mutate(motorist_involved = ifelse(number_of_motorist_injured > 0 | number_of_motorist_killed > 0, 1, 0))


In [16]:
numeric_cols <- names(crash_data[sapply(crash_data, is.numeric)])
length(numeric_cols)

[1] 21

In [17]:
numeric_cols

[1] "X"                             "zip_code"                     
 [3] "latitude"                      "longitude"                    
 [5] "number_of_persons_injured"     "number_of_persons_killed"     
 [7] "number_of_pedestrians_injured" "number_of_pedestrians_killed" 
 [9] "number_of_cyclist_injured"     "number_of_cyclist_killed"     
[11] "number_of_motorist_injured"    "number_of_motorist_killed"    
[13] "collision_id"                  "month"                        
[15] "year"                          "total_injury"                 
[17] "total_casualty"                "total_injuries_deaths"        
[19] "pedestrian_involved"           "cyclist_involved"             
[21] "motorist_involved"

In [18]:
# filling blank borough with random values
unique_values <- unique(crash_data$borough)

# Removing empty strings from unique values
unique_values <- unique_values[unique_values != ""]

# Replacing empty strings with random values from unique values
crash_data$borough[crash_data$borough == ""] <- sample(unique_values, sum(crash_data$borough == ""), replace = TRUE)


In [19]:
unique(crash_data$borough)

[1] "MANHATTAN"     "BRONX"         "BROOKLYN"      "STATEN ISLAND"
[5] "QUEENS"

In [20]:
# let's rename the columns 
crash_data <- crash_data %>% rename(
  accident_factor = grouped_factor_vehicle_contribution,
  total_victims = total_injuries_deaths
)


In [21]:
names(crash_data)

[1] "X"                             "crash_date"                   
 [3] "crash_time"                    "borough"                      
 [5] "zip_code"                      "latitude"                     
 [7] "longitude"                     "number_of_persons_injured"    
 [9] "number_of_persons_killed"      "number_of_pedestrians_injured"
[11] "number_of_pedestrians_killed"  "number_of_cyclist_injured"    
[13] "number_of_cyclist_killed"      "number_of_motorist_injured"   
[15] "number_of_motorist_killed"     "contributing_factor_vehicle_1"
[17] "contributing_factor_vehicle_2" "contributing_factor_vehicle_3"
[19] "collision_id"                  "vehicle_type_code_1"          
[21] "vehicle_type_code_2"           "vehicle_type_code_3"          
[23] "month"                         "year"                         
[25] "total_injury"                  "total_casualty"               
[27] "total_victims"                 "crash_datetime"               
[29] "time_period"                   "Season"                       
[31] "accident_factor"               "vehicle_category"             
[33] "pedestrian_involved"           "cyclist_involved"             
[35] "motorist_involved"

In [22]:
imp_feats <-  c("borough", "month", "time_period", "Season", "accident_factor", 
                    "vehicle_category", "pedestrian_involved", "cyclist_involved", 
                    "motorist_involved", "total_victims")

In [23]:
# Saving data in parquet format for fast operations
# write_parquet(crash_data[,imp_feats], "model_data.parquet")